# Kaggle competition : When bag of words meets bags of popcorn

I followed the official Kaggle tutorial of the competition https://www.kaggle.com/c/word2vec-nlp-tutorial.

In this tutorial competition, we dig a little "deeper" into sentiment analysis. Google's Word2Vec is a deep-learning inspired method that focuses on the meaning of words. Word2Vec attempts to understand meaning and semantic relationships among words. It works in a way that is similar to deep approaches, such as recurrent neural nets or deep neural nets, but is computationally more efficient. This tutorial focuses on Word2Vec for sentiment analysis.

In [23]:
# First you need to install genism and cython(else it will take days rather than minutes!!!!)

#for genism-:
#easy_install -U gensim

#for cython-:
#pip install Cython

In [2]:
import re
import logging
import time

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

from bs4 import BeautifulSoup

import numpy as np
import pandas as pd      

from gensim.models import Word2Vec

import nltk
# nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list

# Part 1 : simple bag of words model : Preprocessing the model

In [6]:
train = pd.read_csv("labelledDataTrain.csv")
del train['Unnamed: 0']
train.head(5)

filename  sentiment                                               text
0   1563.txt          0  This movie is a perfect example of an excellen...
1   1575.txt          1  I first saw this film during and International...
2  655_1.txt          0  I went into this with my hopes up, by twenty m...
3     29.txt          0  I had some expectation for the movie, since it...
4    182.txt          1  This is a great film.<br /><br />I agreed to w...

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

In [8]:
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      BeautifulSoup(train["text"][0], "lxml")  .get_text() )  # The text to search, remove HTML tags
print(letters_only)

This movie is a perfect example of an excellent book getting ruined by a movie  Jacob Have I Loved is quite possibly the worst film that I have ever seen  There is no storyline  plots disappear  and the editing is awful  To top it all off  the music is straight from a synthesizer and sounds unbelievably terrible  Bridget Fonda s acting is decent  but everyone else s acting is totally amateur  I would suggest this movie to someone who is studying to be a producer as a study on how not to produce a movie as it is chock full of bad cut scenes  bad transitions and acting that should have been re shot  Read the book and don t waste your time with this film 


In [12]:
print(stopwords.words("english"))


[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

In [14]:
def review_to_words( raw_review ):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, "lxml").get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

In [16]:
num_reviews = train["text"].size # train is the training data
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
for i in xrange( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d" % ( i+1, num_reviews ))                                                                 
    clean_train_reviews.append( review_to_words( train["text"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 5387
Review 2000 of 5387
Review 3000 of 5387
Review 4000 of 5387
Review 5000 of 5387


# Part 2 : Using Word2Vec to extract features

In [24]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    # Here documents are basically sentences
    # 1. Remove HTML
    review_text = BeautifulSoup(review,"lxml").get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [25]:
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [26]:
%%time
#nltk.download()
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = []  # Initialize an empty list of sentences
print("Parsing sentences from training set")
for i, review in enumerate(train["text"]):    
    sentences += review_to_sentences(review.decode('utf-8'), tokenizer)

/home/amrith/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:198: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/home/amrith/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.happierabroad.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)


Parsing sentences from training set
CPU times: user 25.8 s, sys: 432 ms, total: 26.2 s
Wall time: 26.3 s


In [27]:
# Check how many sentences we have in total - should be around 850,000+
print(len(sentences))

58377


In [28]:
print(sentences[0])
print(sentences[1])

[u'this', u'movie', u'is', u'a', u'perfect', u'example', u'of', u'an', u'excellent', u'book', u'getting', u'ruined', u'by', u'a', u'movie']
[u'jacob', u'have', u'i', u'loved', u'is', u'quite', u'possibly', u'the', u'worst', u'film', u'that', u'i', u'have', u'ever', u'seen']


## Training the model

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

* Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
* Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
* Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
* Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
* Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
* Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
* Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [29]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [30]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [36]:
%%time
# Initialize and train the model (this will take some time)
print("Training model...") # what is word2vec
model = Word2Vec(sentences[:25000], workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


Training model...
CPU times: user 10.6 s, sys: 92 ms, total: 10.7 s
Wall time: 2.97 s


In [37]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

In [39]:
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "Data/Kaggle/300features_40minwords_10context"
model.save(model_name)

## Exploring the Model Results

Congratulations on making it successfully through everything so far! Let's take a look at the model we created out of our 75,000 training reviews.

The "doesnt_match" function will try to deduce which word in a set is most dissimilar from the others:

In [43]:
>>> model.doesnt_match("man woman child kitchen".split())

'child'

In [45]:
>>> model.doesnt_match("paris berlin london austria".split())

'paris'

We can also use the "most_similar" function to get insight into the model's word clusters:

In [46]:
>>> model.most_similar("man")

[(u'woman', 0.9594568014144897),
 (u'young', 0.9456523656845093),
 (u'wife', 0.9332680702209473),
 (u'father', 0.9329059720039368),
 (u'girl', 0.9325944185256958),
 (u'mother', 0.9201096296310425),
 (u'michael', 0.9069634675979614),
 (u'himself', 0.9037177562713623),
 (u'family', 0.8947961330413818),
 (u'finds', 0.8947592377662659)]

In [49]:
>>> model.most_similar("wife")

[(u'father', 0.9758774638175964),
 (u'woman', 0.9640824794769287),
 (u'girl', 0.95993572473526),
 (u'daughter', 0.9577762484550476),
 (u'mother', 0.9532424211502075),
 (u'finds', 0.9484679698944092),
 (u'meets', 0.9412771463394165),
 (u'husband', 0.9411513805389404),
 (u'son', 0.9395575523376465),
 (u'man', 0.9332680702209473)]

In [51]:
>>> model.most_similar("awful")

[(u'terrible', 0.9522275328636169),
 (u'boring', 0.9325945377349854),
 (u'overall', 0.9180511236190796),
 (u'writing', 0.8998401165008545),
 (u'horrible', 0.8967346549034119),
 (u'budget', 0.8760374188423157),
 (u'cool', 0.8738042116165161),
 (u'acting', 0.8734940886497498),
 (u'obvious', 0.8719107508659363),
 (u'quality', 0.8664720058441162)]